In [1]:
import torch

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!nvidia-smi

Sun May  8 11:29:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install transformers

In [5]:
# import re
# import os 
# def rm_tags(text):
#   re_tags = re.compile(r'<[^>]+>')
#   return re_tags.sub(' ',text)

# def read_files(filetype):
#   path = "aclImdb/"
#   file_list=[]
  
#   positive_path = path + filetype + "/pos/"
#   for f in os.listdir(positive_path):
#       file_list += [positive_path + f]
      
#   negative_path = path + filetype + "/neg/"
#   for f in os.listdir(negative_path):
#       file_list += [negative_path + f]   
      
#   print("read",filetype,"files:",len(file_list))
  
#   all_labels = ([1]*12500+[0]*12500)
  
#   all_texts = []
#   for fi in file_list:
#       with open(fi,encoding = 'utf8') as file_input:
#           all_texts += [rm_tags(" ".join(file_input.readlines()))]
          
#   return all_labels,all_texts

# y_train,train_text = read_files("train")
# y_test,test_text = read_files("test")

# 本地路径
# train=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/labeledTrainData.tsv",'\t', error_bad_lines= False)
# test=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/testData.tsv","\t", error_bad_lines=False)
# unlabeled = pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/unlabeledTrainData.tsv", "\t", error_bad_lines=False)
# 在线路径 ../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
import pandas as pd
train=pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",'\t', error_bad_lines= False)
test=pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip","\t", error_bad_lines=False)
unlabeled = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", "\t", error_bad_lines=False)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 43043: expected 2 fields, saw 3\n'


In [6]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [8]:
%%time
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import os
import html
import matplotlib.pyplot as plt
import seaborn as sb

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Phrases
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split


# 文本清洗
train.review=html.unescape(train.review)
test.review=html.unescape(test.review)
# 去除网页标记

train.review=train.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
            .str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
            .str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()
test.review=test.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
            .str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
            .str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()

def remove_emoji(text):
    '''使用正则表达式去除网页标记'''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

train['review']=train['review'].apply(lambda x: remove_emoji(x))
test['review']=test['review'].apply(lambda x: remove_emoji(x))

from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore") # 字符串标准化

train["review"] = train["review"].apply(remove_accent)
test["review"] = test["review"].apply(remove_accent)

CPU times: user 17.2 s, sys: 209 ms, total: 17.4 s
Wall time: 18.1 s


In [9]:
sentences=train['review']

In [10]:
labels=train['sentiment']

In [11]:
test_sentences=train['review']

In [12]:
test_labels=train['sentiment']

In [13]:
# Print the original sentence.
print(' Original: ', sentences[0])

 Original:  with all this stuff going down at the moment with mj ive started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mjs feeling towards the press and also the obvious message of drugs are bad mkaybr br visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of himbr br the actual feature film bit when it finally starts is only o

In [14]:
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

Tokenized:  ['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'm', '##j', 'iv', '##e', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wi', '##z', 'and', 'watched', 'moon', '##walker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moon', '##walker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'm', '##js', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'mk', '##ay', '##br', 'br', 'visually', 'impressive

In [15]:
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Token IDs:  [2007, 2035, 2023, 4933, 2183, 2091, 2012, 1996, 2617, 2007, 1049, 3501, 4921, 2063, 2318, 5962, 2000, 2010, 2189, 3666, 1996, 5976, 4516, 2182, 1998, 2045, 3427, 1996, 15536, 2480, 1998, 3427, 4231, 26965, 2153, 2672, 1045, 2074, 2215, 2000, 2131, 1037, 3056, 12369, 2046, 2023, 3124, 2040, 1045, 2245, 2001, 2428, 4658, 1999, 1996, 27690, 2074, 2000, 2672, 2191, 2039, 2026, 2568, 3251, 2002, 2003, 5905, 2030, 7036, 4231, 26965, 2003, 2112, 8308, 2112, 3444, 2143, 2029, 1045, 3342, 2183, 2000, 2156, 2012, 1996, 5988, 2043, 2009, 2001, 2761, 2207, 2070, 1997, 2009, 2038, 11259, 7696, 2055, 1049, 22578, 3110, 2875, 1996, 2811, 1998, 2036, 1996, 5793, 4471, 1997, 5850, 2024, 2919, 12395, 4710, 19892, 7987, 17453, 8052, 2021, 1997, 2607, 2023, 2003, 2035, 2055, 2745, 4027, 2061, 4983, 2017, 19512, 2066, 1049, 3501, 1999, 4312, 2059, 2017, 2024, 2183, 2000, 5223, 2023, 1998, 2424, 2009, 11771, 2070, 2089, 2655, 1049, 3501, 2019, 13059, 16774, 2005, 9619, 2075, 2000, 1996, 2437, 1

In [16]:
%%time
MAX_LEN=128
input_ids = train['review'].apply(lambda x: tokenizer.encode(x, add_special_tokens =True,max_length=MAX_LEN ))
test_input_ids = test['review'].apply(lambda x: tokenizer.encode(x, add_special_tokens =True,max_length=MAX_LEN ))
# input_ids = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in sentences]
# test_input_ids=[tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in test_sentences]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 6min 6s, sys: 378 ms, total: 6min 6s
Wall time: 6min 7s


In [17]:
%%time
from keras.preprocessing.sequence import pad_sequences
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, 
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          value=0, 
                          truncating="post", 
                          padding="post")

test_input_ids = pad_sequences(test_input_ids, 
                               maxlen=MAX_LEN, 
                               dtype="long", 
                                  value=0, 
                               truncating="post", 
                               padding="post")


Padding token: "[PAD]", ID: 0
CPU times: user 1.2 s, sys: 98.2 ms, total: 1.3 s
Wall time: 1.32 s


In [18]:
%%time 
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

test_attention_masks = []

# For each sentence...
for sent in test_input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    test_attention_masks.append(att_mask)


CPU times: user 4.21 s, sys: 1.01 ms, total: 4.21 s
Wall time: 4.21 s


In [19]:
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, 
                                                                                     labels, 
                                                                                     random_state=42, 
                                                                                     test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       labels,
                                                       random_state=42, 
                                                       test_size=0.2)


In [20]:
train_inputs

array([[  101,  2023,  3185, ...,  2338,  2310,   102],
       [  101,  4830, 14227, ...,  1037,  3110,   102],
       [  101,  2089,  1996, ...,  2003,  2130,   102],
       ...,
       [  101,  2003,  1999, ...,  8327,  1037,   102],
       [  101,  1045,  2371, ...,     0,     0,     0],
       [  101,  1996,  6808, ...,  2431,  2019,   102]])

In [21]:
import torch
train_inputs = torch.LongTensor(train_inputs)

validation_inputs = torch.LongTensor(validation_inputs)
test_inputs=torch.LongTensor(test_input_ids)

In [22]:
train_labels = torch.LongTensor(train_labels.values)

In [23]:
validation_labels = torch.LongTensor(validation_labels.values)

In [24]:
test_labels=torch.LongTensor(test_labels.values)

In [25]:
test_labels

tensor([1, 1, 0,  ..., 0, 0, 1])

In [26]:
train_masks = torch.Tensor(train_masks)
validation_masks = torch.Tensor(validation_masks)
test_masks=torch.Tensor(test_attention_masks)

In [27]:
train_inputs

tensor([[  101,  2023,  3185,  ...,  2338,  2310,   102],
        [  101,  4830, 14227,  ...,  1037,  3110,   102],
        [  101,  2089,  1996,  ...,  2003,  2130,   102],
        ...,
        [  101,  2003,  1999,  ...,  8327,  1037,   102],
        [  101,  1045,  2371,  ...,     0,     0,     0],
        [  101,  1996,  6808,  ...,  2431,  2019,   102]])

In [28]:
train_masks

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [29]:
train_labels

tensor([0, 0, 0,  ..., 1, 1, 0])

In [30]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [31]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
                                                        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
                                                        num_labels = 2, # The number of output labels--2 for binary classification.
                                                                        # You can increase this for multi-class tasks.   
                                                        output_attentions = False, # Whether the model returns attentions weights.
                                                        output_hidden_states = False, # Whether the model returns all hidden-states.
                                                     )
    
# Tell pytorch to run this model on the GPU.
model.cuda()


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [32]:
device='cuda' if torch.cuda.is_available() else 'cpu'
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                                        eps = 1e-8) # args.adam_epsilon  - default is 1e-8.

In [33]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                      num_warmup_steps = 100, # Default value in run_glue.py
                      num_training_steps = total_steps)

In [34]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [36]:
%%time
import random
device='cuda' if torch.cuda.is_available() else 'cpu'
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

CPU times: user 113 µs, sys: 1 µs, total: 114 µs
Wall time: 142 µs


In [37]:
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)

            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  1,250.    Elapsed: 0:00:09.
  Batch    80  of  1,250.    Elapsed: 0:00:18.
  Batch   120  of  1,250.    Elapsed: 0:00:27.
  Batch   160  of  1,250.    Elapsed: 0:00:35.
  Batch   200  of  1,250.    Elapsed: 0:00:44.
  Batch   240  of  1,250.    Elapsed: 0:00:52.
  Batch   280  of  1,250.    Elapsed: 0:01:01.
  Batch   320  of  1,250.    Elapsed: 0:01:10.
  Batch   360  of  1,250.    Elapsed: 0:01:18.
  Batch   400  of  1,250.    Elapsed: 0:01:27.
  Batch   440  of  1,250.    Elapsed: 0:01:35.
  Batch   480  of  1,250.    Elapsed: 0:01:44.
  Batch   520  of  1,250.    Elapsed: 0:01:53.
  Batch   560  of  1,250.    Elapsed: 0:02:01.
  Batch   600  of  1,250.    Elapsed: 0:02:10.
  Batch   640  of  1,250.    Elapsed: 0:02:18.
  Batch   680  of  1,250.    Elapsed: 0:02:27.
  Batch   720  of  1,250.    Elapsed: 0:02:35.
  Batch   760  of  1,250.    Elapsed: 0:02:44.
  Batch   800  of  1,250.    Elapsed: 0:02:53.
  Batch   840  of

In [38]:
t0 = time.time()
model.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1
print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
print("  Test took: {:}".format(format_time(time.time() - t0)))


  Accuracy: 0.4982
  Test took: 0:01:38
